从谷歌云盘进行数据导入

In [ ]:
import os
from google.colab import drive
drive.mount('/content/drive')
PATH = os.path.join('/content/drive/My Drive/', 'ml-latest-small')

读取并查看数据

评级数据文件结构（ratings.csv）
-----------------------------------------

所有评分都包含在“ ratings.csv”文件中。标题行之后的此文件的每一行代表一个用户对一部电影的评分，并具有以下格式：

    userId，movieId，评分，时间戳

该文件中的各行首先由userId排序，然后在用户中由movieId排序。

评分以5星级标准进行，以半星级为增量（0.5星级-5.0星级）。

时间戳表示自1970年1月1日午夜协调世界时（UTC）起的秒数。

In [ ]:
import pandas as pd
import numpy as np
ratings = pd.read_csv(r'/content/drive/My Drive/ml-latest-small/ratings.csv', sep=',', header=0, engine = 'python')
ratings.head()

标签数据文件结构（tags.csv）
-----------------------------------

所有标签都包含在“ tags.csv”文件中。标题行之后的该文件的每一行代表一个用户将一个标签应用于一部电影，并具有以下格式：

    userId，movieId，标记，时间戳

该文件中的各行首先由userId排序，然后在用户中由movieId排序。

标签是用户生成的有关电影的元数据。每个标签通常是一个单词或简短短语。特定标签的含义，值和目的由每个用户确定。

时间戳表示自1970年1月1日午夜协调世界时（UTC）起的秒数。

In [ ]:
import pandas as pd
import numpy as np
tags = pd.read_csv(r'/content/drive/My Drive/ml-latest-small/tags.csv', sep=',', header=0, engine = 'python')
tags.head()

电影数据文件结构（movies.csv）
---------------------------------------

电影信息包含在文件“ movies.csv”中。该文件的标题行之后的每一行代表一部电影，并具有以下格式：

    movieId，标题，流派

电影标题是手动输入的，或者是从<https://www.themoviedb.org/>导入的，并在括号中包含发行年份。这些标题中可能存在错误和不一致之处。

流派是用管道分隔的列表，并从以下列表中选择：

*动作
*冒险
* 动画
*儿童
*喜剧片
*犯罪
*纪录片
*戏剧
*幻想
*黑色电影
*恐怖
*音乐剧
*神秘
*浪漫
* 科幻
*惊悚片
*战争
*西方
*（未列出流派）

In [ ]:
import pandas as pd
import numpy as np
movies = pd.read_csv(r'/content/drive/My Drive/ml-latest-small/movies.csv', sep=',', header=0, engine = 'python')
movies.head()

链接数据文件结构（links.csv）
---------------------------------------

可用于链接到其他电影数据源的标识符包含在“ links.csv”文件中。该文件的标题行之后的每一行代表一部电影，并具有以下格式：

    movieId，imdbId，tmdbId

movieId是<https://movielens.org>使用的电影的标识符。例如，电影《玩具总动员》具有链接<https://movielens.org/movies/1>。

imdbId是<http://www.imdb.com>使用的电影的标识符。例如，电影《玩具总动员》具有链接<http://www.imdb.com/title/tt0114709/>。

tmdbId是<https://www.themoviedb.org>使用的电影的标识符。例如，电影《玩具总动员》具有链接<https://www.themoviedb.org/movie/862>。

上面所列资源的使用受每个提供商的条款约束。

In [ ]:
import pandas as pd
import numpy as np
links = pd.read_csv(r'/content/drive/My Drive/ml-latest-small/links.csv', sep=',', header=0, engine = 'python')
links.head()

对数据进行预处理，生成userid-item矩阵,并将400个用户作为训练集，剩下210个用户作为测试集

In [ ]:
from numpy import array
from scipy import sparse
rate = np.zeros([610, 193609])
for i in range(100836):
  rate[ratings.iloc[i, 0] - 1, ratings.iloc[i, 1] - 1] = ratings.iloc[i, 2]

print(rate, rate.shape)
print(sum(rate[:, 65]))
all_zero = []
for m in range(193609):
  if sum(rate[:, m]) == 0:
    all_zero.append(m)
print(all_zero)
rate = np.delete(rate, all_zero, axis=1)
print(rate, rate.shape)

user_based_cf_2

In [ ]:
#计算两个向量x y的皮尔森相似度,c是物品在user-item矩阵中的平均得分
def person(x,y,c):
    x1 = []
    y1 = []
    for i in range(x.size):
        if(x[i] * y[i] != 0):
            x1.append(x[i] - c[i])
            y1.append(y[i] - c[i])
    x1 = np.array(x1)
    y1 = np.array(y1)
    return x1.dot(y1) / (np.linalg.norm(x1) * np.linalg.norm(y1))

#1.like_users矩阵:
#与 待推荐的用户 最相似的用户
#返回一个向量[[a1 a2 ... ak]
          #[b1, b2, ... bk]
          #[...]]
#表示和第0个待推荐用户最相似的top_k个用户是a1 a2 ...ak用户
def get_mostly_like_items(ui_matrix, train_num, test_num, top_k):
    mean_item_rating = ui_matrix.mean(axis = 0)
    person_matrix = np.zeros((train_num + test_num,train_num + test_num))
    for i in range(train_num , test_num + train_num):
        for  j in range(test_num + train_num):
            if(j != i): person_matrix[i][j] = person(ui_matrix[i],ui_matrix[j],mean_item_rating)
    person_matrix = person_matrix[train_num:,...]
    like_users = np.argsort(person_matrix)[...,ui_matrix.shape[1] - top_k:]# 把得到的皮尔森相似度排序后返回最大的top_k个返回,就是和本用户最相似的用户
    x = 1 - (ui_matrix!= 0)#排除已经打过分的物品
    like_items = np.zeros((test_num, ui_matrix.shape[1]))
    for i in range(test_num):
        sum_weight = 0
        for j in like_users[i]: 
            sum_weight += person_matrix[i][j]
            like_items[i] += x[i+train_num] * (person_matrix[i][j] * ui_matrix[j])
        like_items[i] = like_items[i] / sum_weight
    like_items = np.argsort(like_items)[...,ui_matrix.shape[1] - top_k:]
    return like_items

def user_based_cf(ui_matrix, train_num, test_num, top_k):
    like_items = get_mostly_like_items(ui_matrix, train_num, test_num, top_k)
    for i in range(test_num):
        print("对用户 " + repr(i+train_num) + ", 推荐的物品是: ",like_items[i])
    print(like_items)

top_k = 3 #选前3个最相似的用户
user_based_cf(rate, train_num, test_num, top_k)